# Quantum Teleporation Application

In this demo, we'll demonstrate the simulation of the teleportation application. The network topology will have two nodes: Alice and Bob. Alice will teleport a qubit to Bob. 

## 1. The Basics

The figure below shows the quantum circuit of teleportation, as well as showing which qubits belong to Alice and which qubit belongs to Bob.

<img src="figures/teleport-circuit.png" width="600">

Let's break down quantum teleporation into 4 steps.

Step 1:

<img src="figures/teleport-1.png" width="600">

Step 2:

<img src="figures/teleport-2.png" width="600">

Step 3:

<img src="figures/teleport-3.png" width="600">

Step 4:

<img src="figures/teleport-4.png" width="600">

## 2. The Code